In [ ]:
from math import pi

from hume.qiskit.util import show_reports
from hume.simulator.circuit import QuantumRegister, QuantumCircuit
from hume.utils.common import CONFIG


def add_gate(qc, cs, target, gate, angle):
    if len(cs) == 1:
        gate = 'c' + gate
    elif len(cs) > 1:
        gate = 'mc' + gate
        
    m = getattr(qc, gate)
    
    if angle is None:
        if len(cs) == 1:
            m(cs[0], int(target))
        elif len(cs) > 1:
            m(cs, int(target))
        else:
            m(int(target))
    else:
        if len(cs) == 1:
            m(angle, cs[0], int(target))
        elif len(cs) > 1:
            m(angle, cs, int(target))
        else:
            m(angle, int(target))

In [ ]:
import mercury as mr

app = mr.App(title='Circuit Builder', description='Experimenting with quantum circuits', show_code=False)

In [ ]:

no_arg_gates = ['h', 'x', 'y', 'z']
arg_gates = ['p', 'rx', 'ry', 'rz']
gates = no_arg_gates + arg_gates

control_gates = ['x', 'y', 'z', 'p', 'ry']
m_control_gates = ['x', 'p']

select_str = 'Select...'

qubits = mr.Slider(value=3, min=1, max=5, label='Number of qubits', step=1)
gate = mr.Select(value=select_str, choices=[select_str] + gates, label='Gate')
target = mr.Select(value=select_str, choices=[select_str] + [str(k) for k in range(qubits.value)], label='Target')

control = mr.Select(value=select_str, choices=[select_str] if target.value == select_str else [select_str] + [k for k in range(int(target.value))] + [k for k in range(1+int(target.value), qubits.value)], label='Control',
                          hidden=qubits.value == 1 or target.value == select_str or gate.value not in control_gates or gate.value in m_control_gates)

controls = mr.MultiSelect(value=[], choices=[] if target.value == select_str else [k for k in range(int(target.value))] + [k for k in range(1+int(target.value), qubits.value)], label='Controls', 
                          hidden=qubits.value == 1 or target.value == select_str or gate.value not in m_control_gates)

arg = mr.Numeric(value=0.0, min=-360.0, max=360.0, label='Angle', step = 0.01, 
                         hidden=target.value == select_str or (gate.value not in arg_gates))

report = mr.Checkbox( value=True, label='Create step', disabled=gate.value == select_str or target.value == select_str)
button = mr.Button(label='Add transformation', style="primary", disabled=gate.value == select_str or target.value == select_str)

 

In [ ]:
CONFIG.use_mpl()
n = qubits.value

if 'qc' not in locals() or n != sum(qc.regs):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)   

In [ ]:
if button.clicked:
    add_gate(qc, controls.value if gate.value in m_control_gates else [control.value] if gate.value in control_gates and control.value != select_str else [] , target.value, gate.value, pi/180*arg.value if gate.value in arg_gates else None)
    qc.reports.pop('Step in progress', None)
    if report.value:
        qc.report(f'Step {1+len(qc.reports)}')
    else:
        qc.report('Step in progress')
    show_reports(qc)
elif 'qc' in locals():
    show_reports(qc)